# FCE Processing

In [18]:
from lxml import etree
import pandas as pd
import os
import re
from itertools import zip_longest
import csv
import unicodedata
import thesis_utils
from collections import defaultdict, Counter
import spacy
import nltk
import string
from rapidfuzz.fuzz import ratio

/home/elisabetta/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
comet_ml is installed but `COMET_API_KEY` is not set.
/home/elisabetta/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [19]:
fce_dataset = '/home/elisabetta/Scrivania/thesis/fce-released-dataset/dataset'

In [20]:
multiged_fce_dev =  thesis_utils.read_tsv_file_and_find_sentences_without_headers('./MULTI-GED2023 DATA/en_fce_dev.tsv')
multiged_dev_info2 = thesis_utils.get_list_ids_tokens_gold(multiged_fce_dev)
print(len(multiged_fce_dev))

2191


In [21]:
def clean_multiged(sentences_tokens_lists):
    for sentence in sentences_tokens_lists:
        for i, (idx, token, label) in enumerate(sentence):
            if token.startswith('\\'):
                cleaned_token = token.strip('\\')
                sentence[i] = (idx, cleaned_token, label)
    return sentences_tokens_lists


multiged_dev_info = clean_multiged(multiged_dev_info2)

In [22]:
print(multiged_dev_info[0])

[('T0', '13th', 'c'), ('T1', 'June', 'c'), ('T2', '2000', 'c')]


In [23]:
## ALMOST THERE...

# nlp = spacy.load("en_core_web_md")

# def process_fce_directory_return_sentences_spacy2_ins(input_dir):
#     all_sentences = []
#     token_counter = 0

#     for root_dir, _, files in os.walk(input_dir):
#         for filename in files:
#             if filename.endswith('.xml') and '-checkpoint' not in filename:
#                 filepath = os.path.join(root_dir, filename)

#                 try:
#                     tree = etree.parse(filepath)
#                     root = tree.getroot()
#                     coded_answers = root.findall('.//coded_answer')

#                     for coded in coded_answers:
#                         for p in coded.findall('.//p'):
#                             current_sentence = []

#                             def process_text(text, label, correction='', error_type='', is_correction=False):
#                                 nonlocal current_sentence, token_counter
#                                 if not text or not text.strip():
#                                     return

#                                 doc = nlp(text.strip())
#                                 for token in doc:
#                                     token_counter += 1
#                                     row = [
#                                         filename,
#                                         f"T{token_counter}",      # Only global token counter
#                                         token.text,
#                                         label,
#                                         correction,
#                                         error_type if label == 'i' else ''
#                                     ]
#                                     current_sentence.append(row)
                                    
#                                     if token.text in ['.', '!', '?']:
#                                         all_sentences.append(current_sentence)
#                                         current_sentence = []

#                             def process_element(elem, label='c', correction='', error_type=''):
#                                 nonlocal current_sentence

#                                 if elem.text:
#                                     if label != 'c' or not isinstance(elem.tag, str) or elem.tag != 'c':
#                                         process_text(elem.text, label, correction, error_type)

#                                 for child in elem:
#                                     if child.tag == 'NS':
#                                         etype = child.get('type', '')
#                                         i_elem = child.find('i')
#                                         c_elem = child.find('c')

#                                         if etype.startswith('M') and c_elem is not None:
#                                             correction_text = c_elem.text.strip() if c_elem.text else ''
#                                             process_text(
#                                                 '',
#                                                 'c',  # Mark as correct
#                                                 correction=correction_text,
#                                                 error_type=etype
#                                             )

#                                         if i_elem is not None:
#                                             process_element(
#                                                 i_elem,
#                                                 label='i',
#                                                 correction=c_elem.text if c_elem is not None else '',
#                                                 error_type=etype
#                                             )

#                                     elif child.tag == 'i':
#                                         process_element(child, label='i', correction=correction, error_type=error_type)
#                                     elif child.tag == 'c':
#                                         pass
#                                     else:
#                                         process_element(child, label=label, correction=correction, error_type=error_type)

#                                     if child.tail:
#                                         process_text(child.tail, 'c')

#                             process_element(p)

#                             if current_sentence:
#                                 all_sentences.append(current_sentence)
#                                 current_sentence = []

#                 except etree.XMLSyntaxError as e:
#                     print(f"XML parse error in {filename}: {e}")
#                 except Exception as e:
#                     print(f"Error processing {filename}: {e}")

#     return all_sentences


In [24]:
#PERFETTO!!!

import os
import spacy
from lxml import etree

nlp = spacy.load("en_core_web_md")

def process_fce_directory_return_sentences_spacy2_ins2(input_dir):
    all_sentences = []
    token_counter = 0

    for root_dir, _, files in os.walk(input_dir):
        for filename in files:
            if filename.endswith('.xml') and '-checkpoint' not in filename:
                filepath = os.path.join(root_dir, filename)

                try:
                    tree = etree.parse(filepath)
                    root = tree.getroot()
                    coded_answers = root.findall('.//coded_answer')

                    for coded in coded_answers:
                        for p in coded.findall('.//p'):
                            current_sentence = [] #element

                            def process_text(text, label, correction='', error_type='', is_correction=False):
                                nonlocal current_sentence, token_counter
                                if not text or not text.strip():
                                    return

                                doc = nlp(text.strip())
                                for token in doc:
                                    token_counter += 1
                                    row = [
                                        filename,
                                        f"T{token_counter}",      # Only global token counter
                                        token.text,
                                        label,
                                        correction,
                                        error_type if label == 'i' else ''
                                    ]
                                    current_sentence.append(row)
                                    
                                    if token.text in ['.', '!', '?']:
                                        all_sentences.append(current_sentence)
                                        current_sentence = []

                            def process_element(elem, label='c', correction='', error_type=''):
                                nonlocal current_sentence

                                if elem.text:
                                    if label != 'c' or not isinstance(elem.tag, str) or elem.tag != 'c':
                                        process_text(elem.text, label, correction, error_type)

                                for child in elem:
                                    if child.tag == 'NS':
                                        etype = child.get('type', '')
                                        i_elem = child.find('i')
                                        c_elem = child.find('c')

                                        # Handle Metalinguistic cases with explicit <c>
                                        if etype.startswith('M') and c_elem is not None:
                                            correction_text = c_elem.text.strip() if c_elem.text else ''
                                            process_text(
                                                '',
                                                'c',  # Mark as correct
                                                correction=correction_text,
                                                error_type=etype
                                            )

                                        # Handle standard <i> cases
                                        if i_elem is not None:
                                            process_element(
                                                i_elem,
                                                label='i',
                                                correction=c_elem.text if c_elem is not None else '',
                                                error_type=etype
                                            )

                                        # Handle R-type errors with no <i> or <c>
                                        elif not list(child):  # No children: assume plain incorrect span
                                            process_text(
                                                child.text,
                                                label='i',
                                                correction='',
                                                error_type=etype
                                            )

                                    elif child.tag == 'i':
                                        process_element(child, label='i', correction=correction, error_type=error_type)
                                    elif child.tag == 'c':
                                        pass  # Do not process <c> directly unless in M-type handling
                                    else:
                                        process_element(child, label=label, correction=correction, error_type=error_type)

                                    if child.tail:
                                        process_text(child.tail, 'c')

                            process_element(p)

                            if current_sentence:
                                all_sentences.append(current_sentence)
                                current_sentence = []

                except etree.XMLSyntaxError as e:
                    print(f"XML parse error in {filename}: {e}")
                except Exception as e:
                    print(f"Error processing {filename}: {e}")

    return all_sentences


In [25]:
# import os
# import spacy
# from lxml import etree

# nlp = spacy.load("en_core_web_md")

# def process_fce_directory_return_sentences_spacy2_ins4(input_dir):
#     all_sentences = []
#     token_counter = 0

#     for root_dir, _, files in os.walk(input_dir):
#         for filename in files:
#             if filename.endswith('.xml') and '-checkpoint' not in filename:
#                 filepath = os.path.join(root_dir, filename)

#                 try:
#                     tree = etree.parse(filepath)
#                     root = tree.getroot()
#                     coded_answers = root.findall('.//coded_answer')

#                     for coded in coded_answers:
#                         for p in coded.findall('.//p'):
#                             current_sentence = []

#                             def process_text(text, label, correction='', error_type='', is_correction=False):
#                                 nonlocal current_sentence, token_counter
#                                 if not text or not text.strip():
#                                     return

#                                 doc = nlp(text.strip())
#                                 for token in doc:
#                                     token_counter += 1
#                                     row = [
#                                         filename,
#                                         f"T{token_counter}",      # Only global token counter
#                                         token.text,
#                                         label,
#                                         correction,
#                                         error_type if label == 'i' else ''
#                                     ]
#                                     current_sentence.append(row)

#                                     if token.text in ['.', '!', '?']:
#                                         all_sentences.append(current_sentence)
#                                         current_sentence = []

#                             def process_element(elem, label='c', correction='', error_type=''):
#                                 nonlocal current_sentence

#                                 if elem.text:
#                                     if label != 'c' or not isinstance(elem.tag, str) or elem.tag != 'c':
#                                         process_text(elem.text, label, correction, error_type)

#                                 for child in elem:
#                                     if child.tag == 'NS':
#                                         etype = child.get('type', '')
#                                         i_elem = child.find('i')
#                                         c_elem = child.find('c')

#                                         # Handle M-type corrections with <c> only
#                                         if etype.startswith('M') and c_elem is not None:
#                                             correction_text = c_elem.text.strip() if c_elem.text else ''
#                                             process_text(
#                                                 '',
#                                                 'c',
#                                                 correction=correction_text,
#                                                 error_type=etype
#                                             )

#                                         # Handle standard i+c structure
#                                         if i_elem is not None:
#                                             process_element(
#                                                 i_elem,
#                                                 label='i',
#                                                 correction=c_elem.text if c_elem is not None else '',
#                                                 error_type=etype
#                                             )

#                                         # Handle R-type with no children (<NS>just_text</NS>)
#                                         elif etype.startswith('R') and not list(child):
#                                             process_text(
#                                                 child.text,
#                                                 label='i',
#                                                 correction='',
#                                                 error_type=etype
#                                             )

#                                         else:
#                                             # Recurse into nested <NS> tags (like FN inside RN)
#                                             for nested in child:
#                                                 process_element(
#                                                     nested,
#                                                     label=label,
#                                                     correction=correction,
#                                                     error_type=etype
#                                                 )

#                                     elif child.tag == 'i':
#                                         process_element(child, label='i', correction=correction, error_type=error_type)
#                                     elif child.tag == 'c':
#                                         pass  # Skip <c> unless handled above
#                                     else:
#                                         process_element(child, label=label, correction=correction, error_type=error_type)

#                                     if child.tail:
#                                         process_text(child.tail, 'c')

#                             process_element(p)

#                             if current_sentence:
#                                 all_sentences.append(current_sentence)
#                                 current_sentence = []

#                 except etree.XMLSyntaxError as e:
#                     print(f"XML parse error in {filename}: {e}")
#                 except Exception as e:
#                     print(f"Error processing {filename}: {e}")

#     return all_sentences


In [26]:
sentences_fce_spacy= process_fce_directory_return_sentences_spacy2_ins2(fce_dataset)

In [27]:
print(sentences_fce_spacy[9170])

[['doc450.xml', 'T148778', 'She', 'c', '', ''], ['doc450.xml', 'T148779', 'has', 'c', '', ''], ['doc450.xml', 'T148780', 'not', 'c', '', ''], ['doc450.xml', 'T148781', 'a', 'c', '', ''], ['doc450.xml', 'T148782', 'lot', 'c', '', ''], ['doc450.xml', 'T148783', 'of', 'c', '', ''], ['doc450.xml', 'T148784', 'friend', 'i', 'friends', 'AGN'], ['doc450.xml', 'T148785', 'because', 'c', '', ''], ['doc450.xml', 'T148786', 'of', 'c', '', ''], ['doc450.xml', 'T148787', 'this', 'c', '', ''], ['doc450.xml', 'T148788', 'point', 'i', '', 'RN'], ['doc450.xml', 'T148789', '.', 'c', '', '']]


In [28]:
no_empty_spaces_sentences_fce_spacy = []

for sentence in sentences_fce_spacy:
    cleaned_sentence = []

    for line in sentence:
        token = line[2]  

        if token.strip() == '':
            continue 
        cleaned_sentence.append(line)  

    if cleaned_sentence:
        no_empty_spaces_sentences_fce_spacy.append(cleaned_sentence)


In [29]:
print(len(no_empty_spaces_sentences_fce_spacy))

34415


In [30]:
print(no_empty_spaces_sentences_fce_spacy[3])

[['doc3139.xml', 'T63', 'Our', 'c', '', ''], ['doc3139.xml', 'T64', 'English', 'c', '', ''], ['doc3139.xml', 'T65', 'club', 'c', '', ''], ['doc3139.xml', 'T66', 'is', 'c', '', ''], ['doc3139.xml', 'T67', 'not', 'c', '', ''], ['doc3139.xml', 'T68', 'a', 'c', '', ''], ['doc3139.xml', 'T69', 'very', 'c', '', ''], ['doc3139.xml', 'T70', 'big', 'c', '', ''], ['doc3139.xml', 'T71', 'one', 'c', '', ''], ['doc3139.xml', 'T72', ',', 'c', '', ''], ['doc3139.xml', 'T73', 'with', 'c', '', ''], ['doc3139.xml', 'T74', '14', 'c', '', ''], ['doc3139.xml', 'T75', 'students', 'c', '', ''], ['doc3139.xml', 'T76', 'and', 'c', '', ''], ['doc3139.xml', 'T77', 'a', 'c', '', ''], ['doc3139.xml', 'T78', 'teacher', 'c', '', ''], ['doc3139.xml', 'T79', '.', 'c', '', '']]


In [31]:
# def split_tokens_and_renumber(sentences):
#     processed_sentences = []
#     token_counter = 0  # Global token counter

#     for sentence in sentences:
#         new_sentence = []

#         for row in sentence:
#             file_name = row[0]
#             token_text = row[2]
#             other_fields = row[3:]

#             if '/' in token_text and any(c.isalpha() for c in token_text):
#                 parts = []
#                 current = ''
#                 for char in token_text:
#                     if char == '/':
#                         if current:
#                             parts.append(current)
#                             current = ''
#                         parts.append('/')
#                     else:
#                         current += char
#                 if current:
#                     parts.append(current)

#                 for part in parts:
#                     new_row = [
#                         file_name,
#                         f'T{token_counter}',
#                         part,
#                         *other_fields
#                     ]
#                     new_sentence.append(new_row)
#                     token_counter += 1

#             else:
#                 new_row = [
#                     file_name,
#                     f'T{token_counter}',
#                     token_text,
#                     *other_fields
#                 ]
#                 new_sentence.append(new_row)
#                 token_counter += 1

#         processed_sentences.append(new_sentence)

#     return processed_sentences
# sent_sp2 = split_tokens_and_renumber(no_empty_spaces_sentences_fce_spacy)

In [32]:
def split_tokens_and_renumber(sentences):
    processed_sentences = []

    for sentence in sentences:
        new_sentence = []
        token_counter = 0  # Reset for each sentence

        for row in sentence:
            file_name = row[0]
            token_text = row[2]
            other_fields = row[3:]

            if '/' in token_text and any(c.isalpha() for c in token_text):
                parts = []
                current = ''
                for char in token_text:
                    if char == '/':
                        if current:
                            parts.append(current)
                            current = ''
                        parts.append('/')
                    else:
                        current += char
                if current:
                    parts.append(current)

                for part in parts:
                    new_row = [
                        file_name,
                        f'T{token_counter}',
                        part,
                        *other_fields
                    ]
                    new_sentence.append(new_row)
                    token_counter += 1

            else:
                new_row = [
                    file_name,
                    f'T{token_counter}',
                    token_text,
                    *other_fields
                ]
                new_sentence.append(new_row)
                token_counter += 1

        processed_sentences.append(new_sentence)

    return processed_sentences
sent_sp2 = split_tokens_and_renumber(no_empty_spaces_sentences_fce_spacy)

In [33]:
print(sent_sp2[1])

[['doc3139.xml', 'T0', 'My', 'c', '', ''], ['doc3139.xml', 'T1', 'name', 'c', '', ''], ['doc3139.xml', 'T2', 'is', 'c', '', ''], ['doc3139.xml', 'T3', 'Akihiro', 'c', '', ''], ['doc3139.xml', 'T4', 'and', 'c', '', ''], ['doc3139.xml', 'T5', 'I', 'c', '', ''], ['doc3139.xml', 'T6', "'m", 'c', '', ''], ['doc3139.xml', 'T7', 'a', 'c', '', ''], ['doc3139.xml', 'T8', 'member', 'c', '', ''], ['doc3139.xml', 'T9', 'of', 'c', '', ''], ['doc3139.xml', 'T10', 'an', 'c', '', ''], ['doc3139.xml', 'T11', 'English', 'c', '', ''], ['doc3139.xml', 'T12', 'club', 'c', '', ''], ['doc3139.xml', 'T13', 'at', 'c', '', ''], ['doc3139.xml', 'T14', 'West', 'c', '', ''], ['doc3139.xml', 'T15', 'Isle', 'c', '', ''], ['doc3139.xml', 'T16', 'College', 'c', '', ''], ['doc3139.xml', 'T17', '.', 'c', '', '']]


In [34]:
weird_merged_sentences = []
i = 0
n = len(sent_sp2)

while i < n:
    sentence = sent_sp2[i]

    if (
        len(sentence) == 2
        and sentence[0][2].isdigit()        
        and sentence[1][2] in {'.', ')'} 
    ):
        if i + 1 < n:
            next_sentence = sent_sp2[i + 1]

            weird_merged_sentences.append(sentence)
            weird_merged_sentences.append(next_sentence)

            merged = sentence + next_sentence
            weird_merged_sentences.append(merged)

            i += 2
        else:
            weird_merged_sentences.append(sentence)
            i += 1
    else:
        if sentence[0][2] == '"':
            merged_sentence = previous_sentence + sentence
            original_previous_sentence = previous_sentence
            original_current_sentence = sentence
            stripped_current_sentence = [token for token in sentence if token[2] != '"']

            weird_merged_sentences.append(original_previous_sentence) 
            weird_merged_sentences.append(original_current_sentence)
            weird_merged_sentences.append(merged_sentence)
            weird_merged_sentences.append(stripped_current_sentence)
        else:
            weird_merged_sentences.append(sentence)

        i += 1
    previous_sentence = sentence

In [35]:
print(len(weird_merged_sentences))

35006


In [36]:
filtered_sentences_spacy = [
    sent for sent in weird_merged_sentences
    if not (len(sent) == 1 and sent[0][2] == '.')
]
print(len(filtered_sentences_spacy))
even_more_filtered____ = []
for sent in filtered_sentences_spacy:
    new_sent = [] 
    for i, row in enumerate(sent):
        if i == 0 or row[3] != sent[i-1][5]:
            new_sent.append(row)
    even_more_filtered____.append(new_sent)


34921


In [37]:
# ## take into account '. .'
# punctuation_to_merge = set(string.punctuation) - {"."}

# merged_sentences_spacy_idk = []

# for sent in even_more_filtered____:
#     if len(sent) == 1:
#         token = sent[0][2]

#         if token == "..":
#             if merged_sentences_spacy_idk and merged_sentences_spacy_idk[-1]:
#                 merged_sentences_spacy_idk[-1][-1][2] = "..."
#         elif token in punctuation_to_merge:
#             if merged_sentences_spacy_idk:
#                 merged_sentences_spacy_idk[-1].extend(sent)
#         else:
#             merged_sentences_spacy_idk.append(sent)

#     elif len(sent) == 2:
#         token1 = sent[0][2]
#         token2 = sent[1][2]

#         if token1 == "." and token2 == ".":
#             if merged_sentences_spacy_idk and merged_sentences_spacy_idk[-1]:
#                 merged_sentences_spacy_idk[-1][-1][2] = "..."
#         elif token1 in string.punctuation and token2 in string.punctuation:
#             if merged_sentences_spacy_idk:
#                 merged_sentences_spacy_idk[-1].extend(sent)
#         else:
#             merged_sentences_spacy_idk.append(sent)

#     else:
#         merged_sentences_spacy_idk.append(sent)
        
# print(len(merged_sentences_spacy_idk))

In [38]:
punctuation_to_merge = set(string.punctuation) - {"."}

merged_sentences_spacy_idk = []

for sent in even_more_filtered____:
    if len(sent) == 1:
        token = sent[0][2]

        if token == "..":
            if merged_sentences_spacy_idk and merged_sentences_spacy_idk[-1]:
                merged_sentences_spacy_idk[-1][-1][2] = "..."
        elif token in punctuation_to_merge:
            if merged_sentences_spacy_idk:
                merged_sentences_spacy_idk[-1].extend(sent)
        else:
            merged_sentences_spacy_idk.append(sent)

    elif len(sent) == 2:
        token1 = sent[0][2]
        token2 = sent[1][2]

        if token1 == "." and token2 == ".":
            if merged_sentences_spacy_idk and merged_sentences_spacy_idk[-1]:
                merged_sentences_spacy_idk[-1][-1][2] = "..."
        elif token1 in string.punctuation and token2 in string.punctuation:
            if merged_sentences_spacy_idk:
                merged_sentences_spacy_idk[-1].extend(sent)
        else:
            merged_sentences_spacy_idk.append(sent)

    else:
        new_sent = []
        skip_next = False
        merged_occurred = False

        for i in range(len(sent)):
            if skip_next:
                skip_next = False
                continue

            current_token = sent[i][2]
            if i + 1 < len(sent):
                next_token = sent[i + 1][2]
                if current_token in {"have", "has"} and next_token == "n't":
                    # Merge tokens
                    merged_token = current_token + next_token
                    merged_row = sent[i][:] 
                    merged_row[2] = merged_token 
                    new_sent.append(merged_row)
                    skip_next = True
                    merged_occurred = True
                    continue

            new_sent.append(sent[i])

        merged_sentences_spacy_idk.append(new_sent)

In [39]:
print(len(merged_sentences_spacy_idk))

34713


In [40]:
combined_sentences_2or3 = []
while i < n:
    sentence = merged_sentences_spacy_idk[i]
    combined_sentences_2or3.append(sentence)

    if i + 1 < n:
        merged_2 = sentence + merged_sentences_spacy_idk[i + 1]
        combined_sentences_2or3.append(merged_2)

    i += 1

print(len(combined_sentences_2or3))

68793


In [41]:
for sentence in combined_sentences_2or3:
    for line in sentence:
        token = line[2]
        if token.strip() == '':
            print("Sentence contains a newline or empty token:")
            print(sentence)
            break

In [42]:
def match_fce_to_multiged(sentences_fce, multiged_tokens):
    token_sequence_to_fce_occurrences = defaultdict(list)
    for fce_sentence in sentences_fce:
        token_sequence = tuple(token_info[2] for token_info in fce_sentence)
        token_sequence_to_fce_occurrences[token_sequence].append(fce_sentence)

    token_sequence_to_multiged_occurrences = defaultdict(list)
    for multiged_sent in multiged_tokens:
        token_sequence = tuple(token[1] for token in multiged_sent)
        token_sequence_to_multiged_occurrences[token_sequence].append(multiged_sent)

    matched_sentences = []
    matched_sequences = set()

    for token_sequence, fce_occurrences in token_sequence_to_fce_occurrences.items():
        multiged_occurrences = token_sequence_to_multiged_occurrences.get(token_sequence, [])
        match_count = min(len(fce_occurrences), len(multiged_occurrences))

        for i in range(match_count):
            fce_sentence = fce_occurrences[i]
            multiged_sentence = multiged_occurrences[i]

            if len(fce_sentence) != len(multiged_sentence):
                continue

            fce_with_labels = [
                tuple(token_info) + (multiged_label,)
                for token_info, (_, _, multiged_label) in zip(fce_sentence, multiged_sentence)
            ]

            matched_sentences.append(fce_with_labels)
            matched_sequences.add(token_sequence)

    return matched_sentences, matched_sequences

In [43]:
dev_matches_spacy_idk, dev_sequences_spacy_idk = match_fce_to_multiged(combined_sentences_2or3, multiged_dev_info)

print(f'Number of matched dev sentences: {len(dev_matches_spacy_idk)}')       

all_matched_sequences_spacy = dev_sequences_spacy_idk

unmatched_multiged_dev = []
for sentence in multiged_dev_info:
    token_sequence = tuple(token[1] for token in sentence)
    if token_sequence not in all_matched_sequences_spacy:
        unmatched_multiged_dev.append(sentence)

print(f'Dev sentences still to match: {len(unmatched_multiged_dev)}')

unmatched_sentences_fce = []
for fce_sentence in combined_sentences_2or3:
    token_sequence = tuple(token_info[2] for token_info in fce_sentence)
    if token_sequence not in all_matched_sequences_spacy:
        unmatched_sentences_fce.append(fce_sentence)

print(f"Total unmatched FCE sentences: {len(unmatched_sentences_fce)}")

Number of matched dev sentences: 2149
Dev sentences still to match: 42
Total unmatched FCE sentences: 64379


In [44]:
def extract_sentence_counts(data, index):
    return Counter(tuple(token[index] for token in sentence) for sentence in data)

def compare_sentence_counts(data_matches, data_dev):
    counts_matches = extract_sentence_counts(data_matches, 2)
    counts_dev = extract_sentence_counts(data_dev, 1)

    all_sentences = set(counts_matches.keys()) | set(counts_dev.keys())

    sorted_sentences = sorted(all_sentences)

    print(f"{'Sentence':<60} | {'Matches':<8} | {'Dev':<8}")
    print("-" * 85)
    
    for sent in sorted_sentences:
        matches_count = counts_matches.get(sent, 0)
        dev_count = counts_dev.get(sent, 0)
        if matches_count > 1 or dev_count > 1:
            sent_str = ' '.join(sent)
            print(f"{sent_str:<60} | {matches_count:<8} | {dev_count:<8}")


In [45]:
compare_sentence_counts(dev_matches_spacy_idk, multiged_dev_info)

Sentence                                                     | Matches  | Dev     
-------------------------------------------------------------------------------------
13th June , 2000                                             | 4        | 4       
13th June 2000                                               | 4        | 4       
Best wishes                                                  | 3        | 3       
Competition Organiser                                        | 3        | 3       
Dear Helen ,                                                 | 2        | 2       
Dear Kim                                                     | 3        | 3       
Dear Kim ,                                                   | 11       | 11      
Dear Kim :                                                   | 2        | 2       
Dear Mr Robertson ,                                          | 4        | 4       
Dear Mrs Ryan ,                                              | 4        | 4       
D

In [46]:
# import csv

# headers = ['filename', 'token', 'correction', 'error_type', 'gold_label']

# with open('processed_fce2.tsv', 'w', newline='', encoding='utf-8') as f_out:
#     writer = csv.writer(f_out, delimiter='\t')
#     writer.writerow(headers)

#     for sentence in dev_matches_spacy_idk:
#         for row in sentence:
#             row_without_id = [row[0], row[2], row[4], row[5], row[6]]
#             writer.writerow(row_without_id)
#         writer.writerow([])


In [47]:
#Attempting extra sentences...

In [48]:
def extract_tokens_idx_and_filenames(list_of_lines_for_sentence):
    all_sentences_tok = []
    all_indexes= []
    all_filenames = []
    
    for sent in list_of_lines_for_sentence:
        sent_tok = []
        sent_idx = []
        sent_nam = []
        for line in sent:
            file_name = line[0]
            tok_idx=line[1]
            token = line[2]
            sent_tok.append(token)
            sent_idx.append(tok_idx)
            sent_nam.append(file_name)
        all_sentences_tok.append(sent_tok)
        all_indexes.append(sent_idx)
        all_filenames.append(sent_nam)
    return all_sentences_tok,all_indexes,all_filenames
    
all_unmatched_fce_tok_sent,all_unmatched_fce_indexes, all_unmatched_fce_filenames = extract_tokens_idx_and_filenames(unmatched_sentences_fce)

In [49]:
filtered_unmatched_dev_tok_sent = []
filtered_unmatched_fce_indexes = []
filtered_unmatched_fce_filenames = []

for sentence, idx, name in zip(all_unmatched_fce_tok_sent,all_unmatched_fce_indexes, all_unmatched_fce_filenames):
    if len(sentence) > 0 and len(idx)>0 and len(name) > 0:
        filtered_unmatched_dev_tok_sent.append(sentence)
        filtered_unmatched_fce_indexes.append(idx)
        filtered_unmatched_fce_filenames.append(name)

In [50]:
sentences_as_single_strings_fce = []
prev_cleaned = None
prev_clean_indices = None
prev_filename = None

for sentence, idx_list, filename in zip(filtered_unmatched_dev_tok_sent, filtered_unmatched_fce_indexes, filtered_unmatched_fce_filenames):
    cleaned = ''
    clean_indices = ''
    
    for idx, token in zip(idx_list, sentence):
        clean_token = token.replace('\\\\\\', '')
        cleaned += clean_token
        clean_indices += idx * len(clean_token)

    sentences_as_single_strings_fce.append((filename[0], cleaned, clean_indices))
    
    if prev_cleaned is not None and prev_clean_indices is not None:
        combined_cleaned = prev_cleaned + cleaned
        combined_indices = prev_clean_indices + clean_indices
        sentences_as_single_strings_fce.append((prev_filename[0], combined_cleaned, combined_indices))
    
    prev_cleaned = cleaned
    prev_clean_indices = clean_indices
    prev_filename = filename

    sentences_as_single_strings_fce.append((filename[0], cleaned, clean_indices))

In [51]:
sentences_as_single_strings_multiged_dev = []

for sentence in unmatched_multiged_dev:
    cleaned_sentence = ''
    sentence_indices = ''  
    clean_indices = ''
    for token in sentence:
        token_text = token[1]  
        token_idx = token[0] 
        token_label = token[-1]
        if token_text == '\\"':
            cleaned_sentence += '"'
        else:
            cleaned_sentence += token_text

        sentence_indices += token_idx * len(token_text)
        cleaned_labels = ''.join(token[-1].replace('\\\\\\', '') for token in sentence)
    sentences_as_single_strings_multiged_dev.append((sentence_indices,cleaned_sentence, cleaned_labels))

In [52]:
def split_token_ids(index_string):
    return re.findall(r'T\d+', index_string)

def debug_token_ids(sentence, index_string, label=""):
    token_ids = split_token_ids(index_string)
    print(f"\n--- DEBUG TOKEN IDS ({label}) ---")
    print(f"Sentence length: {len(sentence)}")
    print(f"Index string length: {len(index_string)}")
    print(f"Number of token IDs: {len(token_ids)}")
    print("Sentence (repr):", repr(sentence))
    print("Token IDs (first 50):", token_ids[:50])
    print("--- END DEBUG ---\n")

def get_charwise_token_ids(sentence1, sentence2, indexes1):
    token_ids = split_token_ids(indexes1)

    if len(token_ids) != len(sentence1):
        if sentence1 and sentence1[0] in {' ', '\n'} and len(token_ids) == len(sentence1) - 1:
            sentence1 = sentence1[1:]
        elif sentence1 and sentence1[-1] in {' ', '\n'} and len(token_ids) == len(sentence1) - 1:
            sentence1 = sentence1[:-1]

        if len(token_ids) != len(sentence1):
            debug_token_ids(sentence1, indexes1, label="FCE Sentence Token Alignment")
            raise ValueError(
                f"Mismatch between characters and token IDs: {len(sentence1)} chars vs {len(token_ids)} IDs"
            )

    start = sentence1.find(sentence2)
    if start == -1:
        raise ValueError(
            f"Sentence2 not found in sentence1:\n→ sentence1: {sentence1}\n→ sentence2: {sentence2}"
        )

    end = start + len(sentence2)
    return ''.join(token_ids[start:end])

def find_and_remove_overlapping_matches(not_matching_any, dev_remaining_unmatched, threshold=0.8, normalize=False):
    results = set()
    matched_not_any = set()
    matched_dev = set()

    for file_name, sentence1, indexes1 in not_matching_any:
        found_match = False

        for multi_idx, sentence2, gold_label in dev_remaining_unmatched:
            if sentence2 in matched_dev:
                continue

            if sentence2.strip() == "!":
                continue

            s1 = sentence1.strip().replace('\n', '') if normalize else sentence1
            s2 = sentence2.strip().replace('\n', '') if normalize else sentence2

            similarity = ratio(s1, s2) / 100.0

            if similarity >= threshold:
                if s2 in s1:
                    print(f"\n Match in '{file_name}' — similarity: {round(similarity, 3)}")
                    print(f"→ Sentence 1 (FCE): {s1}")
                    print(f"→ Sentence 2 (MultiGED):  {s2}")

                    try:
                        overlap_index = get_charwise_token_ids(s1, s2, indexes1)
                    except ValueError as e:
                        print(f"Skipping due to index mismatch: {e}")
                        continue

                    results.add((file_name, overlap_index, sentence2, gold_label))
                    matched_not_any.add((file_name, sentence1, indexes1))
                    matched_dev.add(sentence2)
                    found_match = True
                    break  

        if found_match:
            continue

    remaining_not_matching_any = [
        item for item in not_matching_any if item not in matched_not_any
    ]
    remaining_dev = [
        item for item in dev_remaining_unmatched if item[1] not in matched_dev
    ]

    return results, remaining_not_matching_any, remaining_dev


In [53]:
results, remaining_fce, remaining_multiged = find_and_remove_overlapping_matches(
    sentences_as_single_strings_fce,
    sentences_as_single_strings_multiged_dev,
    threshold=0.40
)


 Match in 'doc562.xml' — similarity: 0.85
→ Sentence 1 (FCE): Areyouafraid?Well,youcanlookatyourhouse...Nowadays,wehaveallkindofinstrumentstowashingupthedishes,tocooktobemorecomfortable,forexampletheTV,thevideorecorder,etc.
→ Sentence 2 (MultiGED):  Nowadays,wehaveallkindofinstrumentstowashingupthedishes,tocooktobemorecomfortable,forexampletheTV,thevideorecorder,etc.

 Match in 'doc562.xml' — similarity: 0.822
→ Sentence 1 (FCE): Maybe,inafewyearslaterwe'llhaveabiggerTV,amoresuitableradiobut,frommypointofview,Ithinkthesefactswillnotchangethewayofourlifes.Wewillbeabletoacceleratetheprocessofthecomputer'sgamesforexample...Butnotinthestrangewayyoucanimagine...ImeanIdon'tthinkwe'llchangecarsbyaereoplanes...
→ Sentence 2 (MultiGED):  Maybe,inafewyearslaterwe'llhaveabiggerTV,amoresuitableradiobut,frommypointofview,Ithinkthesefactswillnotchangethewayofourlifes.Wewillbeabletoacceleratetheprocessofthecomputer'sgamesforexample...

 Match in 'doc562.xml' — similarity: 0.475
→ Sentence 1 (FCE): W

In [54]:
print(len(results))
list_res= list(results)

13


In [55]:
def process_data(data_list):
    output = []

    for data in data_list:
        filename, indices, sentence, gold_labels = data

        numeric_indices = []
        i = 0
        while i < len(indices):
            if indices[i] == 'T':
                i += 1
                num_str = ''
                while i < len(indices) and indices[i].isdigit():
                    num_str += indices[i]
                    i += 1
                if num_str:
                    numeric_indices.append(num_str)
            else:
                i += 1

        if not numeric_indices:
            continue

        grouped_indices = []
        current_group = [numeric_indices[0]]
        for num in numeric_indices[1:]:
            if num == current_group[-1]:
                current_group.append(num)
            else:
                grouped_indices.append(current_group)
                current_group = [num]
        grouped_indices.append(current_group)

        if len(grouped_indices) != len(gold_labels):
            print(f"Skipping {filename} - Group/Label mismatch "
                  f"({len(grouped_indices)} vs {len(gold_labels)})")
            continue

        total_span = sum(len(group) for group in grouped_indices)
        if total_span != len(sentence):
            print(f"Skipping {filename} - Span mismatch: sentence len {len(sentence)}, total span {total_span}")
            continue

        start = 0
        word_index_pairs = []

        for idx, group in enumerate(grouped_indices):
            word_length = len(group)
            word = sentence[start:start + word_length]
            unique_index = f"T{group[0]}"
            label = gold_labels[idx]
            word_index_pairs.append([unique_index, word, label])
            start += word_length

        output.append({filename: word_index_pairs})

    return output


In [56]:
processed_matches2 =process_data(results)
print(len(processed_matches2))

Skipping doc857.xml - Group/Label mismatch (20 vs 19)
Skipping doc190.xml - Group/Label mismatch (12 vs 11)
11


In [96]:
all_ = []

for pr_sentence in processed_matches2:
    for pr_filename, pr_lines in pr_sentence.items():
        pr_keys = {(tid, tok) for tid, tok, _ in pr_lines}
        pr_labels = {(tid, tok): label for tid, tok, label in pr_lines}

        matched = False 

        for dev_sentence in unmatched_sentences_fce:
            if not dev_sentence:
                continue
            if dev_sentence[0][0] != pr_filename:
                continue

            dev_keys = {(tok[1], tok[2]) for tok in dev_sentence}

            if not pr_keys.issubset(dev_keys):
                continue

            updated_sentence = []
            for tok in dev_sentence:
                tid, token = tok[1], tok[2]
                label = pr_labels.get((tid, token), '')  
                updated_sentence.append(tok + [label])

            all_.append(updated_sentence)
            matched = True
            break  

        if not matched:
            print(f"No full match found for: {pr_filename} with tokens {pr_keys}")

In [98]:
filtered_data_final = []
filtered_data_final2 = []


for sentence in all_:
    seen_rows = set()
    filtered_sentence = []
    for row in sentence:
        if row[-1] != '':
            row_tuple = tuple(row)
            if row_tuple not in seen_rows:
                filtered_sentence.append(row)
                seen_rows.add(row_tuple)
    filtered_data_final.append(filtered_sentence)


for sentence in filtered_data_final:
    if len(sentence) >= 2 and sentence[0][2] == 'the' and sentence[1][2] == 'Two':
        filtered_data_final2.append(sentence[1:])
    else:
        filtered_data_final2.append(sentence)

In [100]:
filtered_data_final2

[[['doc2302.xml', 'T0', 'It', 'c', '', '', 'c'],
  ['doc2302.xml', 'T1', 'is', 'c', '', '', 'c'],
  ['doc2302.xml', 'T2', 'called', 'c', '', '', 'c'],
  ['doc2302.xml', 'T3', ',', 'i', '"', 'RP', 'c'],
  ['doc2302.xml', 'T4', ',', 'i', '"', 'RP', 'c'],
  ['doc2302.xml', 'T5', 'The', 'c', '', '', 'c'],
  ['doc2302.xml', 'T6', 'London', 'c', '', '', 'c'],
  ['doc2302.xml', 'T7', 'Fashion', 'c', '', '', 'c'],
  ['doc2302.xml', 'T8', 'and', 'c', '', '', 'c'],
  ['doc2302.xml', 'T9', 'Leisure', 'c', '', '', 'c'],
  ['doc2302.xml', 'T10', 'Show', 'c', '', '', 'c'],
  ['doc2302.xml', 'T11', '.', 'c', '', '', 'c'],
  ['doc2302.xml', 'T0', '"', 'c', '', '', 'c']],
 [['doc1797.xml', 'T0', 'Two', 'c', '', '', 'c'],
  ['doc1797.xml', 'T1', 'days', 'c', '', '', 'c'],
  ['doc1797.xml', 'T2', 'later', 'c', '', '', 'c'],
  ['doc1797.xml', 'T3', 'in', 'c', '', '', 'c'],
  ['doc1797.xml', 'T4', 'the', 'c', '', '', 'c'],
  ['doc1797.xml', 'T5', 'morning', 'c', '', '', 'c'],
  ['doc1797.xml', 'T6', ',', '

In [102]:
dev_matches_spacy_idk.extend(filtered_data_final2)

In [104]:
print(len(dev_matches_spacy_idk))

2160


In [106]:
all_matched_sequences = set(
    tuple(token[2] for token in sentence)
    for sentence in dev_matches_spacy_idk
)
final_unmatched_multiged_dev = []
for sentence in multiged_dev_info:
    token_sequence = tuple(token[1] for token in sentence)
    if token_sequence not in all_matched_sequences:
        final_unmatched_multiged_dev.append(sentence)

print(f"Dev sentences still to match: {len(final_unmatched_multiged_dev)}")

new_all_matched=dev_matches_spacy_idk
new_unmatched_sentences_fce = []
for fce_sentence in combined_sentences_2or3:
    token_sequence = tuple(token_info[3] for token_info in fce_sentence)
    if token_sequence not in new_all_matched:
        new_unmatched_sentences_fce.append(fce_sentence)

print(f"Total unmatched FCE sentences: {len(new_unmatched_sentences_fce)}")

Dev sentences still to match: 31
Total unmatched FCE sentences: 68793


In [48]:
dev_matches_spacy_idk[500:555]

[[('doc2231.xml', 'T0', 'Although', 'c', '', '', 'c'),
  ('doc2231.xml', 'T1', 'in', 'c', '', '', 'c'),
  ('doc2231.xml', 'T2', 'those', 'c', '', '', 'c'),
  ('doc2231.xml', 'T3', 'days', 'c', '', '', 'c'),
  ('doc2231.xml', 'T4', 'the', 'c', '', '', 'c'),
  ('doc2231.xml', 'T5', 'only', 'c', '', '', 'c'),
  ('doc2231.xml', 'T6', 'way', 'c', '', '', 'c'),
  ('doc2231.xml', 'T7', 'of', 'c', '', '', 'c'),
  ('doc2231.xml', 'T8', 'learning', 'c', '', '', 'c'),
  ('doc2231.xml', 'T9', 'was', 'c', '', '', 'c'),
  ('doc2231.xml', 'T10', 'reading', 'c', '', '', 'c'),
  ('doc2231.xml', 'T11', '.', 'c', '', '', 'c')],
 [('doc2231.xml', 'T0', 'These', 'c', '', '', 'c'),
  ('doc2231.xml', 'T1', 'days', 'c', '', '', 'c'),
  ('doc2231.xml', 'T2', 'we', 'c', '', '', 'c'),
  ('doc2231.xml', 'T3', 'can', 'c', '', '', 'c'),
  ('doc2231.xml', 'T4', 'use', 'c', '', '', 'c'),
  ('doc2231.xml', 'T5', 'computer', 'c', '', '', 'i'),
  ('doc2231.xml', 'T6', ',', 'c', '', '', 'c'),
  ('doc2231.xml', 'T7', 'tel

In [108]:
for sentence in final_unmatched_multiged_dev:
    print(sentence)
    print()

#4 cases do not match because the sentence is a single token !

[('T0', 'Pat', 'c'), ('T1', 'saw', 'c'), ('T2', 'it', 'c'), ('T3', '!', 'c')]

[('T0', 'The', 'c'), ('T1', 'second', 'c'), ('T2', 'day', 'c'), ('T3', 'objective', 'i'), ('T4', 'was', 'c'), ('T5', 'to', 'c'), ('T6', 'install', 'c'), ('T7', 'the', 'c'), ('T8', 'scene', 'i'), ('T9', ',', 'c'), ('T10', 'with', 'c'), ('T11', 'lights', 'c'), ('T12', ',', 'c'), ('T13', 'loudspeakers', 'c'), ('T14', 'and', 'c'), ('T15', 'all', 'c'), ('T16', 'stuff', 'i'), ('T17', '.', 'c')]

[('T0', 'I', 'c'), ('T1', 'noticed', 'c'), ('T2', 'that', 'c'), ('T3', 'the', 'c'), ('T4', 'artists', 'c'), ('T5', 'were', 'c'), ('T6', 'from', 'c'), ('T7', 'only', 'c'), ('T8', 'six', 'c'), ('T9', 'countries', 'c'), ('T10', 'insted', 'i'), ('T11', 'that', 'c'), ('T12', 'from', 'c'), ('T13', 'around', 'c'), ('T14', 'the', 'c'), ('T15', 'world', 'c'), ('T16', '.', 'c')]

[('T0', 'I', 'c'), ('T1', 'was', 'c'), ('T2', 'promised', 'c'), ('T3', 'that', 'c'), ('T4', 'after', 'c'), ('T5', 'show', 'i'), ('T6', 'I', 'c'), ('T7', 'c

In [433]:
target_tokens = {'business','hand','On'}

for idx, sentence in enumerate(new_unmatched_sentences_fce):
    tokens_in_sentence = {row[2] for row in sentence}
    if target_tokens.issubset(tokens_in_sentence):
        print(f"Found both in sentence {idx}:")
        for r in sentence:
            print(r)
        print()


Found both in sentence 17187:
['doc283.xml', 'T0', 'Firs', 'i', 'First', 'SX']
['doc283.xml', 'T1', 'of', 'c', '', '']
['doc283.xml', 'T2', 'all', 'c', '', '']
['doc283.xml', 'T3', 'Internet', 'c', '', '']
['doc283.xml', 'T4', 'gives', 'c', '', '']
['doc283.xml', 'T5', 'you', 'c', '', '']
['doc283.xml', 'T6', 'an', 'i', '', 'UD']
['doc283.xml', 'T7', 'access', 'c', '', '']
['doc283.xml', 'T8', 'to', 'c', '', '']
['doc283.xml', 'T9', 'a', 'c', '', '']
['doc283.xml', 'T10', 'great', 'c', '', '']
['doc283.xml', 'T11', 'number', 'c', '', '']
['doc283.xml', 'T12', 'of', 'c', '', '']
['doc283.xml', 'T13', 'different', 'c', '', '']
['doc283.xml', 'T14', 'information', 'i', '', 'RN']
['doc283.xml', 'T15', '.', 'c', '', '']
['doc283.xml', 'T0', 'On', 'c', '', '']
['doc283.xml', 'T1', 'the', 'c', '', '']
['doc283.xml', 'T2', 'one', 'c', '', '']
['doc283.xml', 'T3', 'hand', 'c', '', '']
['doc283.xml', 'T4', 'Internet', 'c', '', '']
['doc283.xml', 'T5', 'is', 'c', '', '']
['doc283.xml', 'T6', 'oft

In [103]:
# headers = ['filename', 'token', 'correction', 'error_type', 'gold_label']

# with open('processed_fce.tsv', 'w', newline='', encoding='utf-8') as f_out:
#     writer = csv.writer(f_out, delimiter='\t')
#     writer.writerow(headers)

#     for sentence in dev_matches_spacy_idk:
#         for row in sentence:
#             row_without_id = [row[0], row[2], row[4], row[5], row[6]]
#             writer.writerow(row_without_id)
#         writer.writerow([])